In [ ]:
import pandas as pd
import numpy as np
import base64
import re
from catboost import CatBoostClassifier
from bs4 import BeautifulSoup

In [ ]:
from sklearn import metrics
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split,\
                                    cross_validate, KFold,\
                                    cross_val_score
from sklearn.preprocessing import StandardScaler
from functools import reduce

https://www.kaggle.com/c/antispam-infopoisk/data

In [ ]:
train = pd.read_csv('kaggle_train_data_tab_new.csv', sep = '\t') 

In [ ]:
test = pd.read_csv('kaggle_test_data_tab_new.csv', sep = '\t')

In [ ]:
my_train = pd.DataFrame(train)
my_train.drop(columns = {'Id', 'PageBase64'}, inplace = True)

In [ ]:
my_test = pd.DataFrame(test)
my_test.drop(columns = {'Id', 'Prediction', 'PageBase64'}, inplace = True)

In [ ]:
my_train = pd.read_csv('final_train.csv')

In [ ]:
my_test = pd.read_csv('final_test.csv')

In [ ]:
def validation(X_train, train_target, model, params, folds_gen_func, folds_num=5, thresholds=[0.32], **kwargs):
    
    scaler = StandardScaler()
    scaler.fit(X_train) 
    
    main_res = []
    for param_set in combinations(params):
        
        print(param_set)        
        exact_model = model(**param_set) 
        
        fold_generator = folds_gen_func(folds_num)
        
        for th in thresholds:
#           th = 0.27
            print('th = ', th)
            res = []
            for train_index, test_index in fold_generator.split(X_train, train_target, **kwargs):

                exact_model.fit(scaler.transform(X_train[train_index]), train_target[train_index])

                y_pred = [0 if val < th else 1 for val in exact_model.predict_proba(scaler.transform(X_train[test_index]))[:,1]]

                score = metrics.f1_score(train_target[test_index],\
                                         y_pred, average = 'weighted')
    #               print('threshold = {}, score = {}'.format(th, score))

                res.append(score)
        #                                       exact_model.predict(scaler.transform(X_train[test_index]))))


            mean = sum(res)/len(res)
            print(mean)
#             print('threshold = {}, score = {}'.format(th, mean))
            main_res.append((mean, param_set, th))
    
    best = main_res[np.argmax([res[0] for res in main_res])]
    print('--------max-------')
    print(best)
    
    return best

In [ ]:
def combinations(params):
    
    list_ = []
    for value in params.values():
        list_.append(value)

    tmp = reduce(lambda x, y: list_concat(x, y), list_)

    res = []
    for elem in tmp:
        param_list = flatten(elem)
        param_dict = dict(zip(params.keys(), param_list))
        res.append(param_dict)
        
    return res

In [ ]:
def list_concat(list1, list2):
    
    len1 = len(list1)
    len2 = len(list2)
    
    return [[list1[i], list2[j]] for i in range(len1) for j in range(len2)]

In [ ]:
def flatten(x):
    
    result = []
    for elem in x:
        if hasattr(elem, "__iter__") and not isinstance(elem, str):
            result.extend(flatten(elem))
        else:
            result.append(elem)
            
    return result

In [ ]:
def porn(html):
    global j
    
    rus_spam = ['''18+''']
    
    eng_spam = ['''18+''']
    
    word_counter = 0
    txt = ''
    soup = BeautifulSoup(html, 'html')
    for i in soup.find_all(['a', 'h1', 'title']):
        txt  += i.text + ' '
    
    for i in rus_spam:
        word_counter += len(re.findall(i, txt.lower()))
    
    for i in eng_spam:
        word_counter += len(re.findall(i, txt.lower()))

    j += 1
    if not j % 100:
        print(j, 'loaded')
    
    return word_counter

In [ ]:
 def url_len(url):
        smth = re.search('://[^/]*/', url)
        if type(smth) == re.Match:
            u_l = len(smth.group()[3:-1])
        else:
            u_l = 15
            
        return u_l

In [ ]:
def good_site(url):
    
    if (url.count('.ru') + url.count('.com') > 0):
        return 1
    
    return 0

In [ ]:
def url_words(url):
    key_words = ['''18+''']
    for i in key_words:
        if url.lower().count(i):
            return 1
    return 0

In [ ]:
def len_tags(html, tags):
    global j
    
    len_ = 0
    soup = BeautifulSoup(html, 'html')
    
    for i in soup.find_all(tags):
        len_ += len(i.text)
    
    j += 1
    if not j % 100:
        print(j, 'loaded')
    return len_

In [ ]:
def body_len(html):
    global j
    
    soup = BeautifulSoup(html, 'html')
    
    j += 1
    if not j % 100:
        print(j, 'loaded')
    
    return len(re.findall('\w+',(soup.get_text())))

In [ ]:
def tag_counter(html, tag):
    global j 
    
    soup = BeautifulSoup(html, 'html')
        
    j += 1
    if not j % 100:
        print(j, 'loaded')
        
    return len(soup.find_all(tag))

In [ ]:
%%time
# Переводим из Base64 в raw_html
my_train['raw_html'] =\
            train['PageBase64'].apply(lambda x: base64.b64decode(x))
my_test['raw_html'] =\
            test['PageBase64'].apply(lambda x: base64.b64decode(x))

CPU times: user 5.73 s, sys: 338 ms, total: 6.06 s
Wall time: 6.12 s


In [ ]:
%%time
# Количество картинок
j = 0
my_train['img'] =\
            my_train['raw_html'].apply(lambda x: tag_counter(str(x),'img'))
my_test['img'] =\
            my_test['raw_html'].apply(lambda x: tag_counter(str(x),'img'))

In [ ]:
save_tables()
# my_train['url_words']

In [ ]:
%%time
# Количество b
j = 0
my_train['b'] =\
            my_train['raw_html'].apply(lambda x: tag_counter(str(x),'b'))
my_test['b'] =\
            my_test['raw_html'].apply(lambda x: tag_counter(str(x),'b'))

In [ ]:
%%time
# Количество a
j = 0
my_train['a'] =\
            my_train['raw_html'].apply(lambda x: tag_counter(str(x),'a'))
my_test['a'] =\
            my_test['raw_html'].apply(lambda x: tag_counter(str(x),'a'))


In [ ]:
%%time
# Количество br
j = 0
my_train['br'] =\
            my_train['raw_html'].apply(lambda x: tag_counter(str(x),'br'))
my_test['br'] =\
            my_test['raw_html'].apply(lambda x: tag_counter(str(x),'br'))

In [ ]:
%%time
# Количество table
j = 0
my_train['table'] =\
            my_train['raw_html'].apply(lambda x: tag_counter(str(x),'table'))
my_test['table'] =\
            my_test['raw_html'].apply(lambda x: tag_counter(str(x),'table'))

In [ ]:
%%time
# Количество span
j = 0
my_train['span'] =\
            my_train['raw_html'].apply(lambda x: tag_counter(str(x),'span'))
my_test['span'] =\
            my_test['raw_html'].apply(lambda x: tag_counter(str(x),'span'))

In [ ]:
%%time
# Количество tr
j = 0
my_train['tr'] =\
            my_train['raw_html'].apply(lambda x: tag_counter(str(x),'tr'))
my_test['tr'] =\
            my_test['raw_html'].apply(lambda x: tag_counter(str(x),'tr'))

In [ ]:
%%time
# Длина заголовков
j = 0
my_train['title_len'] =\
            my_train['raw_html'].apply(lambda x: len_tags(x,['title', 'h1']))
my_test['title_len'] =\
            my_test['raw_html'].apply(lambda x: len_tags(x,['title', 'h1']))

In [ ]:
%%time
# Длина текста в ссылках
j = 0
my_train['a_len'] =\
            my_train['raw_html'].apply(lambda x: len_tags(x,['a']))
my_test['a_len'] =\
            my_test['raw_html'].apply(lambda x: len_tags(x,['a']))

In [ ]:
%%time
# Длина текста
j = 0
my_train['body_len'] =\
            my_train['raw_html'].apply(lambda x: body_len(x))
my_test['body_len'] =\
            my_test['raw_html'].apply(lambda x: body_len(x))

In [ ]:
%%time
#Длина сырого HTML 
my_train['html_len'] =\
            my_train['raw_html'].apply(lambda x: len(x))
my_test['html_len'] =\
            my_test['raw_html'].apply(lambda x: len(x))

CPU times: user 19 ms, sys: 0 ns, total: 19 ms
Wall time: 18.6 ms


In [ ]:
#Длина текстовой части URL
my_train['url_len'] =\
                my_train['Url'].apply(lambda x: url_len(x))
my_test['url_len'] =\
                my_test['Url'].apply(lambda x: url_len(x))

In [ ]:
#Длина сырого URL
my_train['raw_url_len'] =\
                my_train['Url'].apply(lambda x: len(x))
my_test['raw_url_len'] =\
                my_test['Url'].apply(lambda x: len(x))

In [ ]:
#Первичные признаки спама 18+
my_train['good'] =\
                my_train['Url'].apply(lambda x: good_site(x))
my_test['good'] =\
                my_test['Url'].apply(lambda x: good_site(x))

In [ ]:
#Вхождение ключевых слов в url
my_train['url_words'] =\
                my_train['Url'].apply(lambda x: url_words(x))
my_test['url_words'] =\
                my_test['Url'].apply(lambda x: url_words(x))

In [ ]:
%%time
# Количество популярных 18+ слов
j = 0
my_train['porn']=\
            my_train['raw_html'].apply(lambda x: porn(x))
my_test['porn']=\
            my_test['raw_html'].apply(lambda x: porn(x))

In [ ]:
y_train = np.array(my_train['Prediction'])

f1 = np.array(my_train['img'])
f2 = np.array(my_train['b'])
f3 = np.array(my_train['a'])
f4 = np.array(my_train['br'])
f5 = np.array(my_train['table'])
f6 = np.array(my_train['span'])
f7 = np.array(my_train['tr'])
f8 = np.array(my_train['title_len'])
f9 = np.array(my_train['a_len'])
f10 = np.array(my_train['body_len'])
f11 = np.array(my_train['html_len'])
f12 = np.array(my_train['url_len'])
f13 = np.array(my_train['raw_url_len'])
f14 = np.array(my_train['good'])
f15 = np.array(my_train['url_words'])
f16 = np.array(my_train['porn'])
# my_train

In [ ]:
X_train_1 = np.array([f1, f2, f3, f4, f5, f6, f7, f8, f9, f10,\
                      f11, f12, f13, f14, f15, f16]).T

In [ ]:
f1t = np.array(my_test['img'])
f2t = np.array(my_test['b'])
f3t = np.array(my_test['a'])
f4t = np.array(my_test['br'])
f5t = np.array(my_test['table'])
f6t = np.array(my_test['span'])
f7t = np.array(my_test['tr'])
f8t = np.array(my_test['title_len'])
f9t = np.array(my_test['a_len'])
f10t = np.array(my_test['body_len'])
f11t = np.array(my_test['html_len'])
f12t = np.array(my_test['url_len'])
f13t = np.array(my_test['raw_url_len'])
f14t = np.array(my_test['good'])
f15t = np.array(my_test['url_words'])
f16t = np.array(my_test['porn'])

In [ ]:
X_test = np.array([f1t, f2t, f3t, f4t, f5t, f6t, f7t, f8t, f9t, f10t,\
                   f11t, f12t, f13t, f14t, f15t, f16t]).T

In [ ]:
def save_tables():
    my_train.to_csv('final_train.csv', index=False)
    my_test.to_csv('final_test.csv', index=False)

In [ ]:
params = {'learning_rate':[0.07],\
          'n_estimators' : [2000]}
th = [0.51]

In [ ]:
# best = validation(X_train_1, y_train,\
#                   GradientBoostingClassifier,\
#                   params, KFold, thresholds=th)

In [ ]:
best = validation(X_train_1, y_train,\
                  CatBoostClassifier,\
                  params, KFold, thresholds=th)

In [ ]:
y_pred = predict(X_train_1, X_test, y_train,\
                 CatBoostClassifier, StandardScaler,\
                 best[2], **best[1],)

In [ ]:
submit(y_pred)

In [ ]:
def submit(y_pred):
    submission = pd.DataFrame({'Id' : ['Id'],
                               'Prediction' : np.int64(y_pred)})
    submission.to_csv('submission.csv', index = False)

In [ ]:
def predict(X_train, X_test, train_target, model, scaler=None, th=0.30, **kwargs):
    
    curr_model = model(**kwargs)
    
    if scaler is not None:
        
        your_scaler = scaler()
        your_scaler.fit(X_train)
        X_train = your_scaler.transform(X_train)
        X_test = your_scaler.transform(X_test)
        
    curr_model.fit(X_train, train_target)
#     y_pred = curr_model.predict(X_test)
#     th = 0.35
    print('Threshold = ', th )#,'Params = ', *kwargs.values)
    y_pred = [0 if val < th else 1 for val in curr_model.predict_proba(X_test)[:,1]]
    return y_pred